In [1]:
# Import libraries
import pandas as pd
import numpy as np
import json
from Preprocessing import one_hot_encoding, feature_engineering, promising_transformations, standarization
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import classification_report
import optuna.integration.lightgbm as opt_lgb
from lightgbm import LGBMClassifier
from sklearn.metrics import f1_score
from mlxtend.classifier import StackingCVClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import early_stopping
from lightgbm import log_evaluation
import optuna
import joblib

#### Data Load

In [2]:
df_train = pd.read_csv('../Source/Raw/supply_chain_train.csv')
df_test = pd.read_csv('../Source/Raw/supply_chain_test.csv')

#### Preprocessing

In [3]:
# Columns selections
target = 'Attrition_Flag'
ignore_features = ['CLIENTNUM']
train_idx = ['train_idx']
test_idx = ['test_idx']

In [4]:
categorical_features = df_train.select_dtypes(include=['object']).columns
numeric_features = df_train.select_dtypes(include=['number']).columns.drop([target]+ignore_features+ train_idx)

In [5]:
df_train_targets = df_train[target]
df_train.drop([target]+ignore_features+train_idx, axis=1, inplace=True)

In [6]:
test_ids = df_test.test_idx
df_test.drop(ignore_features+test_idx, axis=1, inplace=True)

In [7]:
# Split data
train_data, val_data, train_target, val_target = train_test_split(df_train, df_train_targets, test_size=0.3, random_state=42)

In [8]:
# Feature engineering
train_data = feature_engineering(train_data, numeric_features)
val_data = feature_engineering(val_data, numeric_features)
test_data = feature_engineering(df_test, numeric_features)

In [9]:
# Promising Transformations
train_data['Credit_Limit_log_sqrt'] = promising_transformations(train_data)
val_data['Credit_Limit_log_sqrt'] = promising_transformations(val_data)
test_data['Credit_Limit_log_sqrt'] = promising_transformations(test_data)

In [10]:
# One-hot encoding for categorical features
train_data = one_hot_encoding(train_data, categorical_features)
val_data = one_hot_encoding(val_data, categorical_features)
test_data = one_hot_encoding(test_data, categorical_features)

In [11]:
# Standarization
train_data, val_data, test_data = standarization(train_data, val_data, test_data)

### Modeling

#### Optuna Settings

In [12]:
# Optuna hyperparameters of LGBMClassifier
def objective(trial):
    params = {
        'learning_rate': trial.suggest_uniform('learning_rate', 0.01, 0.1),
        'num_boost_round': trial.suggest_int('num_boost_round', 100, 1000, step=100),
        'num_leaves': trial.suggest_int('num_leaves', 20, 100),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.5, 1),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 5, 20)
    }

    lgbm_callback = [log_evaluation(0),early_stopping(1000,verbose=False)]
    lgbm = LGBMClassifier(n_jobs=-1, **params, callback=lgbm_callback)
    score = 0
    for train_idx, val_idx in StratifiedKFold(n_splits=5, shuffle=True).split(train_data, train_target):
        train_data_fold = train_data[train_idx]
        train_target_fold = train_target.iloc[train_idx]
        val_data_fold = train_data[val_idx]
        val_target_fold = train_target.iloc[val_idx]
        lgbm.fit(train_data_fold, train_target_fold, eval_set=[(val_data_fold, val_target_fold)], callbacks=lgbm_callback)
        score += f1_score(val_target_fold, lgbm.predict(val_data_fold))
    return score/5


study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

params1 = study.best_params

[I 2023-04-13 06:00:12,334] A new study created in memory with name: no-name-194b551c-9e21-4100-93b7-88f4992e6806
C:\Users\Jon\AppData\Local\Temp\ipykernel_11344\1293619546.py:4: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate': trial.suggest_uniform('learning_rate', 0.01, 0.1),
C:\Users\Jon\AppData\Local\Temp\ipykernel_11344\1293619546.py:7: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'feature_fraction': trial.suggest_uniform('feature_fraction', 0.5, 1),
e:\Software\anaconda3\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in par

[LightGBM] [Warning] at is set=, at= will be ignored. Current value: at=
[LightGBM] [Warning] Unknown parameter: 0x0000015B2C2C6CB0>,<function
[LightGBM] [Warning] Unknown parameter: early_stopping.<locals>._callback
[LightGBM] [Warning] Unknown parameter: callback
[LightGBM] [Warning] Unknown parameter: log_evaluation.<locals>._callback
[LightGBM] [Warning] Unknown parameter: at
[LightGBM] [Warning] Unknown parameter: 0x0000015B3462F5B0>
[LightGBM] [Warning] feature_fraction is set=0.7604388670518838, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7604388670518838
[LightGBM] [Warning] min_data_in_leaf is set=19, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=19
[LightGBM] [Warning] num_iterations is set=900, num_boost_round=900 will be ignored. Current value: num_iterations=900


e:\Software\anaconda3\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] at is set=, at= will be ignored. Current value: at=
[LightGBM] [Warning] Unknown parameter: 0x0000015B2C2C6CB0>,<function
[LightGBM] [Warning] Unknown parameter: early_stopping.<locals>._callback
[LightGBM] [Warning] Unknown parameter: callback
[LightGBM] [Warning] Unknown parameter: log_evaluation.<locals>._callback
[LightGBM] [Warning] Unknown parameter: at
[LightGBM] [Warning] Unknown parameter: 0x0000015B3462F5B0>
[LightGBM] [Warning] feature_fraction is set=0.7604388670518838, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7604388670518838
[LightGBM] [Warning] min_data_in_leaf is set=19, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=19
[LightGBM] [Warning] num_iterations is set=900, num_boost_round=900 will be ignored. Current value: num_iterations=900


e:\Software\anaconda3\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


...

e:\Software\anaconda3\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] at is set=, at= will be ignored. Current value: at=
[LightGBM] [Warning] Unknown parameter: early_stopping.<locals>._callback
[LightGBM] [Warning] Unknown parameter: callback
[LightGBM] [Warning] Unknown parameter: log_evaluation.<locals>._callback
[LightGBM] [Warning] Unknown parameter: at
[LightGBM] [Warning] Unknown parameter: 0x0000015B3462F7F0>,<function
[LightGBM] [Warning] Unknown parameter: 0x0000015B3462F6D0>
[LightGBM] [Warning] feature_fraction is set=0.5783973483262734, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5783973483262734
[LightGBM] [Warning] min_data_in_leaf is set=11, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=11
[LightGBM] [Warning] num_iterations is set=300, num_boost_round=300 will be ignored. Current value: num_iterations=300


e:\Software\anaconda3\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] at is set=, at= will be ignored. Current value: at=
[LightGBM] [Warning] Unknown parameter: early_stopping.<locals>._callback
[LightGBM] [Warning] Unknown parameter: callback
[LightGBM] [Warning] Unknown parameter: log_evaluation.<locals>._callback
[LightGBM] [Warning] Unknown parameter: at
[LightGBM] [Warning] Unknown parameter: 0x0000015B3462F7F0>,<function
[LightGBM] [Warning] Unknown parameter: 0x0000015B3462F6D0>
[LightGBM] [Warning] feature_fraction is set=0.5783973483262734, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5783973483262734
[LightGBM] [Warning] min_data_in_leaf is set=11, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=11
[LightGBM] [Warning] num_iterations is set=300, num_boost_round=300 will be ignored. Current value: num_iterations=300


[I 2023-04-13 06:15:53,464] Trial 49 finished with value: 0.9824295328581638 and parameters: {'learning_rate': 0.06564172695425954, 'num_boost_round': 300, 'num_leaves': 31, 'feature_fraction': 0.5783973483262734, 'min_data_in_leaf': 11}. Best is trial 37 with value: 0.9843245007617705.


In [13]:
# Optuna hyperparameters of CatBoostClassifier
def objective(trial):
    params = {
        "iterations": trial.suggest_int("iterations", 100, 1000),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1),
        "depth": trial.suggest_int("depth", 3, 9),
        "l2_leaf_reg": trial.suggest_int("l2_leaf_reg", 1, 10),
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0, 1),
        "random_seed": 42,
        "eval_metric": "AUC",
        "loss_function": "Logloss",
        "od_type": "Iter",
        "od_wait": 20,
        "verbose": False
    }
    cat = CatBoostClassifier(**params)
    score = 0
    for train_idx, val_idx in StratifiedKFold(n_splits=5, shuffle=True).split(train_data, train_target):
        train_data_fold = train_data[train_idx]
        train_target_fold = train_target.iloc[train_idx]
        val_data_fold = train_data[val_idx]
        val_target_fold = train_target.iloc[val_idx]
        cat.fit(train_data_fold, train_target_fold, eval_set=[(val_data_fold, val_target_fold)])
        score += f1_score(val_target_fold, cat.predict(val_data_fold))
    return score/5

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

params2 = study.best_params

[I 2023-04-13 06:15:53,585] A new study created in memory with name: no-name-3cc9e7ce-a7ba-45b4-8df2-ba7df07a07db
[I 2023-04-13 06:16:43,178] Trial 0 finished with value: 0.9823964487619335 and parameters: {'iterations': 812, 'learning_rate': 0.02783742166148078, 'depth': 7, 'l2_leaf_reg': 4, 'bagging_temperature': 0.9782428163167519}. Best is trial 0 with value: 0.9823964487619335.
[I 2023-04-13 06:17:09,986] Trial 1 finished with value: 0.9816629243463533 and parameters: {'iterations': 285, 'learning_rate': 0.027885410273281154, 'depth': 6, 'l2_leaf_reg': 1, 'bagging_temperature': 0.24997156433892953}. Best is trial 0 with value: 0.9823964487619335.
[I 2023-04-13 06:17:27,085] Trial 2 finished with value: 0.9676540174548542 and parameters: {'iterations': 110, 'learning_rate': 0.013970658619905752, 'depth': 7, 'l2_leaf_reg': 3, 'bagging_temperature': 0.8251818321131604}. Best is trial 0 with value: 0.9823964487619335.
[I 2023-04-13 06:19:05,666] Trial 3 finished with value: 0.98167375

In [14]:
# Optuna hyperparameters of XGBClassifier

def objective(trial):
    params = {
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1),
        "max_depth": trial.suggest_int("max_depth", 3, 9),
        "subsample": trial.suggest_float("subsample", 0.5, 1),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1),
        "gamma": trial.suggest_float("gamma", 0, 1),
        "reg_alpha": trial.suggest_float("reg_alpha", 0, 1),
        "reg_lambda": trial.suggest_float("reg_lambda", 0, 1),
        "n_estimators": 1000,
        "random_state": 42,
        "objective": "binary:logistic"
    }
    
    xgb = XGBClassifier(random_state=42, verbosity=0)
    score = 0
    for train_idx, val_idx in StratifiedKFold(n_splits=5, shuffle=True).split(train_data, train_target):
        train_data_fold = train_data[train_idx]
        train_target_fold = train_target.iloc[train_idx]
        val_data_fold = train_data[val_idx]
        val_target_fold = train_target.iloc[val_idx]
        xgb.fit(train_data_fold, train_target_fold, eval_set=[(val_data_fold, val_target_fold)])
        score += f1_score(val_target_fold, xgb.predict(val_data_fold))
    return score/5

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

params3 = study.best_params

[I 2023-04-13 06:42:26,720] A new study created in memory with name: no-name-81250d31-dd94-4c28-ac41-af3b108d7269


[0]	validation_0-logloss:0.48462
[1]	validation_0-logloss:0.36877
[2]	validation_0-logloss:0.29411
[3]	validation_0-logloss:0.24227
[4]	validation_0-logloss:0.20631
[5]	validation_0-logloss:0.18210
[6]	validation_0-logloss:0.16374
[7]	validation_0-logloss:0.15308
[8]	validation_0-logloss:0.14368
[9]	validation_0-logloss:0.13694
[10]	validation_0-logloss:0.13224
[11]	validation_0-logloss:0.12554
[12]	validation_0-logloss:0.12074
[13]	validation_0-logloss:0.11896
[14]	validation_0-logloss:0.11689
[15]	validation_0-logloss:0.11538
[16]	validation_0-logloss:0.11335
[17]	validation_0-logloss:0.11223
[18]	validation_0-logloss:0.11061
[19]	validation_0-logloss:0.10883
[20]	validation_0-logloss:0.10627
[21]	validation_0-logloss:0.10331
[22]	validation_0-logloss:0.10310
[23]	validation_0-logloss:0.10365
[24]	validation_0-logloss:0.10363
[25]	validation_0-logloss:0.10288
[26]	validation_0-logloss:0.10336
[27]	validation_0-logloss:0.10297
[28]	validation_0-logloss:0.10321
[29]	validation_0-loglos

[I 2023-04-13 06:42:51,594] Trial 0 finished with value: 0.9813946752442689 and parameters: {'learning_rate': 0.06526955614296591, 'max_depth': 5, 'subsample': 0.9601104199795133, 'colsample_bytree': 0.7646821315266774, 'gamma': 0.28505552707397086, 'reg_alpha': 0.029955841069192246, 'reg_lambda': 0.3201348094290284}. Best is trial 0 with value: 0.9813946752442689.


[0]	validation_0-logloss:0.48376
[1]	validation_0-logloss:0.36848
[2]	validation_0-logloss:0.29608
[3]	validation_0-logloss:0.24260
[4]	validation_0-logloss:0.20916
[5]	validation_0-logloss:0.18431
[6]	validation_0-logloss:0.16381
[7]	validation_0-logloss:0.15125
[8]	validation_0-logloss:0.14103
[9]	validation_0-logloss:0.13388
[10]	validation_0-logloss:0.12659
[11]	validation_0-logloss:0.11975
[12]	validation_0-logloss:0.11543
[13]	validation_0-logloss:0.11179
[14]	validation_0-logloss:0.10788
[15]	validation_0-logloss:0.10255
[16]	validation_0-logloss:0.09919
[17]	validation_0-logloss:0.09664
[18]	validation_0-logloss:0.09512
[19]	validation_0-logloss:0.09416
[20]	validation_0-logloss:0.09356
[21]	validation_0-logloss:0.09265
[22]	validation_0-logloss:0.09240
[23]	validation_0-logloss:0.09137
[24]	validation_0-logloss:0.09009
[25]	validation_0-logloss:0.08821
[26]	validation_0-logloss:0.08610
[27]	validation_0-logloss:0.08621
[28]	validation_0-logloss:0.08617
[29]	validation_0-loglos

[I 2023-04-13 06:43:12,489] Trial 1 finished with value: 0.9824152970761204 and parameters: {'learning_rate': 0.013904752277893966, 'max_depth': 7, 'subsample': 0.9713077604936986, 'colsample_bytree': 0.6120151382677268, 'gamma': 0.03270076705006186, 'reg_alpha': 0.039572712627989004, 'reg_lambda': 0.17208831313745954}. Best is trial 1 with value: 0.9824152970761204.


...

[I 2023-04-13 06:58:22,306] Trial 47 finished with value: 0.9834849629439993 and parameters: {'learning_rate': 0.04594812561889069, 'max_depth': 7, 'subsample': 0.8542804958409799, 'colsample_bytree': 0.8250901324501303, 'gamma': 0.2910998093651561, 'reg_alpha': 0.32889444653778493, 'reg_lambda': 0.12006798990667186}. Best is trial 41 with value: 0.9836921711078034.


[0]	validation_0-logloss:0.48517
[1]	validation_0-logloss:0.36983
[2]	validation_0-logloss:0.29484
[3]	validation_0-logloss:0.24265
[4]	validation_0-logloss:0.20862
[5]	validation_0-logloss:0.18304
[6]	validation_0-logloss:0.16511
[7]	validation_0-logloss:0.14877
[8]	validation_0-logloss:0.13959
[9]	validation_0-logloss:0.12903
[10]	validation_0-logloss:0.12168
[11]	validation_0-logloss:0.11286
[12]	validation_0-logloss:0.10860
[13]	validation_0-logloss:0.10471
[14]	validation_0-logloss:0.10223
[15]	validation_0-logloss:0.09629
[16]	validation_0-logloss:0.09250
[17]	validation_0-logloss:0.09061
[18]	validation_0-logloss:0.08875
[19]	validation_0-logloss:0.08877
[20]	validation_0-logloss:0.08725
[21]	validation_0-logloss:0.08637
[22]	validation_0-logloss:0.08472
[23]	validation_0-logloss:0.08402
[24]	validation_0-logloss:0.08354
[25]	validation_0-logloss:0.08390
[26]	validation_0-logloss:0.08365
[27]	validation_0-logloss:0.08230
[28]	validation_0-logloss:0.08050
[29]	validation_0-loglos

[I 2023-04-13 06:58:42,582] Trial 48 finished with value: 0.9822116976134285 and parameters: {'learning_rate': 0.046991950706331095, 'max_depth': 6, 'subsample': 0.8603491658974428, 'colsample_bytree': 0.8152702250123661, 'gamma': 0.2679027130467883, 'reg_alpha': 0.2725733829228833, 'reg_lambda': 0.1039277862106821}. Best is trial 41 with value: 0.9836921711078034.


[0]	validation_0-logloss:0.48991
[1]	validation_0-logloss:0.37533
[2]	validation_0-logloss:0.30226
[3]	validation_0-logloss:0.25443
[4]	validation_0-logloss:0.21956
[5]	validation_0-logloss:0.19202
[6]	validation_0-logloss:0.17457
[7]	validation_0-logloss:0.15940
[8]	validation_0-logloss:0.14854
[9]	validation_0-logloss:0.14090
[10]	validation_0-logloss:0.13594
[11]	validation_0-logloss:0.13033
[12]	validation_0-logloss:0.12637
[13]	validation_0-logloss:0.12358
[14]	validation_0-logloss:0.12069
[15]	validation_0-logloss:0.11814
[16]	validation_0-logloss:0.11539
[17]	validation_0-logloss:0.11447
[18]	validation_0-logloss:0.11387
[19]	validation_0-logloss:0.11356
[20]	validation_0-logloss:0.11217
[21]	validation_0-logloss:0.11077
[22]	validation_0-logloss:0.10857
[23]	validation_0-logloss:0.10841
[24]	validation_0-logloss:0.10838
[25]	validation_0-logloss:0.10801
[26]	validation_0-logloss:0.10593
[27]	validation_0-logloss:0.10505
[28]	validation_0-logloss:0.10385
[29]	validation_0-loglos

[I 2023-04-13 06:59:02,718] Trial 49 finished with value: 0.9832739246817118 and parameters: {'learning_rate': 0.05442984710311894, 'max_depth': 5, 'subsample': 0.8749542174474392, 'colsample_bytree': 0.8699454687455752, 'gamma': 0.1444752810288193, 'reg_alpha': 0.36035367476160657, 'reg_lambda': 0.03578366992389098}. Best is trial 41 with value: 0.9836921711078034.


#### Individual model training with best parameters

In [15]:
# LGBMClassifier Model training
lgbm = LGBMClassifier(n_jobs=-1, **params1)
lgbm.fit(train_data, train_target)


e:\Software\anaconda3\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] feature_fraction is set=0.6066234572454202, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6066234572454202
[LightGBM] [Warning] min_data_in_leaf is set=13, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=13
[LightGBM] [Warning] num_iterations is set=600, num_boost_round=600 will be ignored. Current value: num_iterations=600


LGBMClassifier(feature_fraction=0.6066234572454202,
               learning_rate=0.06616993073754968, min_data_in_leaf=13,
               num_boost_round=600, num_leaves=20)

In [16]:
# CatBoostClassifier Model training
cat = CatBoostClassifier(**params2)
cat.fit(train_data, train_target)

0:	learn: 0.6336295	total: 41.7ms	remaining: 26.7s
1:	learn: 0.5744086	total: 76.2ms	remaining: 24.4s
2:	learn: 0.5229518	total: 114ms	remaining: 24.2s
3:	learn: 0.4789599	total: 154ms	remaining: 24.5s
4:	learn: 0.4409455	total: 189ms	remaining: 24s
5:	learn: 0.4078524	total: 222ms	remaining: 23.5s
6:	learn: 0.3784847	total: 257ms	remaining: 23.3s
7:	learn: 0.3521829	total: 297ms	remaining: 23.5s
8:	learn: 0.3273310	total: 331ms	remaining: 23.3s
9:	learn: 0.3032473	total: 367ms	remaining: 23.1s
10:	learn: 0.2908032	total: 402ms	remaining: 23s
11:	learn: 0.2783977	total: 437ms	remaining: 22.9s
12:	learn: 0.2678231	total: 473ms	remaining: 22.9s
13:	learn: 0.2588123	total: 510ms	remaining: 22.8s
14:	learn: 0.2463539	total: 545ms	remaining: 22.8s
15:	learn: 0.2384749	total: 583ms	remaining: 22.8s
16:	learn: 0.2302461	total: 624ms	remaining: 22.9s
17:	learn: 0.2227896	total: 659ms	remaining: 22.8s
18:	learn: 0.2168653	total: 694ms	remaining: 22.7s
19:	learn: 0.2091759	total: 731ms	remaining

In [17]:
# XGBClassifier Model training
xgb = XGBClassifier(n_jobs=-1, random_state=42, **params3)
xgb.fit(train_data, train_target)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.7425175977742078, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.5176647896222193, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.05941908099348217, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=7, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=-1, num_parallel_tree=None,
              predictor=None, random_state=42, ...)

### Stacking best models

In [18]:
# StackingCVClassifier Model training
stack = StackingCVClassifier(classifiers=[lgbm, cat, xgb], meta_classifier=LGBMClassifier(n_jobs=-1))

### Meta Model metrics

In [19]:
# StackingCVClassifier Model evaluation F1
stack.fit(train_data, train_target)

e:\Software\anaconda3\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
e:\Software\anaconda3\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] feature_fraction is set=0.6066234572454202, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6066234572454202
[LightGBM] [Warning] min_data_in_leaf is set=13, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=13
[LightGBM] [Warning] num_iterations is set=600, num_boost_round=600 will be ignored. Current value: num_iterations=600
0:	learn: 0.6327596	total: 33.5ms	remaining: 21.4s
1:	learn: 0.5766938	total: 67.5ms	remaining: 21.6s
2:	learn: 0.5288713	total: 99.5ms	remaining: 21.2s
3:	learn: 0.4913699	total: 132ms	remaining: 21s
4:	learn: 0.4541101	total: 164ms	remaining: 20.9s
5:	learn: 0.4243416	total: 197ms	remaining: 20.8s
6:	learn: 0.3977601	total: 230ms	remaining: 20.9s
7:	learn: 0.3736083	total: 265ms	remaining: 21s
8:	learn: 0.3473701	total: 303ms	remaining: 21.3s
9:	learn: 0.3199598	total: 338ms	remaining: 21.3s
10:	learn: 0.2977592	total: 383ms	remaining: 21.9s
11:	learn: 0.2850395	total: 426ms	remaining: 22.4s
12:	

e:\Software\anaconda3\lib\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] feature_fraction is set=0.6066234572454202, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6066234572454202
[LightGBM] [Warning] min_data_in_leaf is set=13, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=13
[LightGBM] [Warning] num_iterations is set=600, num_boost_round=600 will be ignored. Current value: num_iterations=600
0:	learn: 0.6336295	total: 43.2ms	remaining: 27.6s
1:	learn: 0.5744086	total: 158ms	remaining: 50.6s
2:	learn: 0.5229518	total: 220ms	remaining: 46.8s
3:	learn: 0.4789599	total: 258ms	remaining: 41.1s
4:	learn: 0.4409455	total: 295ms	remaining: 37.5s
5:	learn: 0.4078524	total: 335ms	remaining: 35.4s
6:	learn: 0.3784847	total: 373ms	remaining: 33.8s
7:	learn: 0.3521829	total: 410ms	remaining: 32.4s
8:	learn: 0.3273310	total: 446ms	remaining: 31.3s
9:	learn: 0.3032473	total: 486ms	remaining: 30.7s
10:	learn: 0.2908032	total: 522ms	remaining: 29.9s
11:	learn: 0.2783977	total: 558ms	remaining: 29.2s
12

StackingCVClassifier(classifiers=[LGBMClassifier(feature_fraction=0.6066234572454202,
                                                 learning_rate=0.06616993073754968,
                                                 min_data_in_leaf=13,
                                                 num_boost_round=600,
                                                 num_leaves=20),
                                  XGBClassifier(base_score=None, booster=None,
                                                callbacks=None,
                                                colsample_bylevel=None,
                                                colsample_bynode=None,
                                                colsample_by...
                                                interaction_constraints=None,
                                                learning_rate=0.05941908099348217,
                                                max_bin=None,
                                                max_

In [20]:
# predict on validation set with StackingCVClassifier model
val_pred = stack.predict(val_data)

In [21]:
# Metrics evaluation F1
score = f1_score(val_target, val_pred)
print(f'F1 score of StackingCVClassifier model: {score}')

F1 score of StackingCVClassifier model: 0.9843411226210552


In [22]:
# Metrics report
print(classification_report(val_target, val_pred))

              precision    recall  f1-score   support

           0       0.92      0.90      0.91       360
           1       0.98      0.99      0.98      2071

    accuracy                           0.97      2431
   macro avg       0.95      0.94      0.95      2431
weighted avg       0.97      0.97      0.97      2431



### Models Export

In [23]:
# Saving models
joblib.dump(stack, 'models/stacked_model.pkl')
joblib.dump(lgbm, 'models/lgbm_model.pkl')
joblib.dump(cat, 'models/cat_model.pkl')
joblib.dump(xgb, 'models/xgb_model.pkl')

['models/xgb_model.pkl']

### Prediction on test dataframe and create submission file

In [24]:
# predict on test set with StackingCVClassifier model
test_pred = stack.predict(test_data)

In [25]:
# Create submission file
predictions = pd.DataFrame({'test_idx':  test_ids, 'Attrition_Flag': test_pred})

In [26]:
predictions['test_idx'] = predictions['test_idx'].astype(str)

In [27]:
jsondata = {}
jsondata['target'] = predictions['Attrition_Flag'].to_dict()

In [28]:
import json

destination='../Source/Predictions/predictions.json'
saveFile = open(destination, 'w')
saveFile.write(json.dumps(jsondata))
saveFile.close()